
<div dir=rtl>
 مثال هایی برای نحوه نمایش سیگما و انجام عملیات های مربوط به آن   </div>

In [1]:
import itertools
import numpy as np
import pyomo.environ as pyo
m1 = pyo.ConcreteModel()

In [2]:
m1.gas = pyo.Set(initialize=('gas1','gas2','gas3'))
m1.oil = pyo.Set(initialize=('oil1','oil2','oil3'))

In [3]:
m1.oil.pprint()

oil : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    3 : {'oil1', 'oil2', 'oil3'}


### Parameters

In [4]:
# m1.gas_profit     = pyo.Param(m1.gas, initialize = {'gas1':70,    'gas2':60,   'gas3':50}  )
# m1.oil_cost       = pyo.Param(m1.oil, initialize = {'oil1':45,    'oil2':35,   'oil3':25}  )
# m1.demands        = pyo.Param(m1.gas, initialize = {'gas1':3000,  'gas2':2000, 'gas3':1000})
# m1.oilpurchase    = pyo.Param(m1.oil, initialize = {'oil1':5000,  'oil2':5000, 'oil3':5000})
# m1.oktan_portion  = pyo.Param(m1.oil, initialize = {'oil1':12,    'oil2':6,    'oil3':8}   )
# m1.solfor_portion = pyo.Param(m1.oil, initialize = {'oil1':0.005, 'oil2':0.02, 'oil3':0.03})
# m1.oktan_need     = pyo.Param(m1.gas, initialize = {'gas1':10,    'gas2':8,    'gas3':6}   )
# m1.solfor_need    = pyo.Param(m1.gas, initialize = {'gas1':0.01,  'gas2':0.02, 'gas3':0.01})


### Variables

In [5]:
m1.x = pyo.Var(m1.oil, m1.gas , domain = pyo.NonNegativeReals)
m1.x.pprint()

x : Size=9, Index=x_index
    Key              : Lower : Value : Upper : Fixed : Stale : Domain
    ('oil1', 'gas1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil1', 'gas2') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil1', 'gas3') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil2', 'gas1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil2', 'gas2') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil2', 'gas3') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil3', 'gas1') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil3', 'gas2') :     0 :  None :  None : False :  True : NonNegativeReals
    ('oil3', 'gas3') :     0 :  None :  None : False :  True : NonNegativeReals


### Objective function

$$
Income  = \sum_{j=1}^3 gp_j \sum_{i=1}^{3} x_{ij} = \sum_{j=1}^3 \sum_{i=1}^{3} gp_j x_{ij} \\
$$

In [6]:
m1.gas_profit = pyo.Param(m1.gas , initialize={'gas1':70,'gas2':60,'gas3':50})
income = sum(m1.gas_profit[j] * m1.x[i,j] for j in m1.gas for i in m1.oil)
print(income)

70*x[oil1,gas1] + 70*x[oil2,gas1] + 70*x[oil3,gas1] + 60*x[oil1,gas2] + 60*x[oil2,gas2] + 60*x[oil3,gas2] + 50*x[oil1,gas3] + 50*x[oil2,gas3] + 50*x[oil3,gas3]


$$
Cost_1  = \sum_{i=1}^{3} oc_i \sum_{j=1}^{3} x_{ij} = \sum_{i=1}^{3} \sum_{j=1}^{3} oc_i x_{ij}
$$

In [7]:
m1.oil_cost = pyo.Param(m1.oil , initialize={'oil1':45,'oil2':35,'oil3':25} )
cost_1 = sum (m1.oil_cost[i] * m1.x[i,j] for i in m1.oil for j in m1.gas)
print(cost_1)

45*x[oil1,gas1] + 45*x[oil1,gas2] + 45*x[oil1,gas3] + 35*x[oil2,gas1] + 35*x[oil2,gas2] + 35*x[oil2,gas3] + 25*x[oil3,gas1] + 25*x[oil3,gas2] + 25*x[oil3,gas3]


$$
Cost_2  = 4 \sum_{i=1}^{3} \sum_{j=1}^{3} x_{ij} 
$$

In [8]:
cost_2 = 4 * sum(m1.x[i,j] for i in m1.oil for j in m1.gas)
print(cost_2)

4*(x[oil1,gas1] + x[oil1,gas2] + x[oil1,gas3] + x[oil2,gas1] + x[oil2,gas2] + x[oil2,gas3] + x[oil3,gas1] + x[oil3,gas2] + x[oil3,gas3])


In [9]:
m1.obj = pyo.Objective(expr = income - cost_1 - cost_2,
                       sense = pyo.maximize)
m1.obj.pprint()

obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 70*x[oil1,gas1] + 70*x[oil2,gas1] + 70*x[oil3,gas1] + 60*x[oil1,gas2] + 60*x[oil2,gas2] + 60*x[oil3,gas2] + 50*x[oil1,gas3] + 50*x[oil2,gas3] + 50*x[oil3,gas3] - (45*x[oil1,gas1] + 45*x[oil1,gas2] + 45*x[oil1,gas3] + 35*x[oil2,gas1] + 35*x[oil2,gas2] + 35*x[oil2,gas3] + 25*x[oil3,gas1] + 25*x[oil3,gas2] + 25*x[oil3,gas3]) - 4*(x[oil1,gas1] + x[oil1,gas2] + x[oil1,gas3] + x[oil2,gas1] + x[oil2,gas2] + x[oil2,gas3] + x[oil3,gas1] + x[oil3,gas2] + x[oil3,gas3])


### Constraints

$$
\sum_{i=1}^{3} x_{ij} = d_j \quad \quad \forall j=1,...,3
$$

In [10]:
m1.demands = pyo.Param(m1.gas , initialize={'gas1':3000,'gas2':2000,'gas3':1000})
def gas_demand (model, j):
    return sum(model.x[i,j] for i in model.oil) == model.demands[j]
m1.c1 = pyo.Constraint(m1.gas , rule = gas_demand)
m1.c1.pprint()

c1 : Size=3, Index=gas, Active=True
    Key  : Lower  : Body                                       : Upper  : Active
    gas1 : 3000.0 : x[oil1,gas1] + x[oil2,gas1] + x[oil3,gas1] : 3000.0 :   True
    gas2 : 2000.0 : x[oil1,gas2] + x[oil2,gas2] + x[oil3,gas2] : 2000.0 :   True
    gas3 : 1000.0 : x[oil1,gas3] + x[oil2,gas3] + x[oil3,gas3] : 1000.0 :   True


In [11]:
# def gas_demand_test (model, j):
#     return sum(model.x[i,j] for i in model.oil) == model.demands[j]
# print(gas_demand_test(m1, j='gas3'))

$$
\sum_{j=1}^{3} x_{ij} \leq p_i \quad \quad \forall i=1,...,3
$$

In [12]:
m1.oil_purchase = pyo.Param(m1.oil , initialize = {'oil1':5000, 'oil2':5000 , 'oil3':5000})

def oil_purchase(model, i):
    return sum(model.x[i,j] for j in model.gas) <= model.oil_purchase[i]
m1.c2 = pyo.Constraint(m1.oil , rule = oil_purchase)
m1.c2.pprint()

c2 : Size=3, Index=oil, Active=True
    Key  : Lower : Body                                       : Upper  : Active
    oil1 :  -Inf : x[oil1,gas1] + x[oil1,gas2] + x[oil1,gas3] : 5000.0 :   True
    oil2 :  -Inf : x[oil2,gas1] + x[oil2,gas2] + x[oil2,gas3] : 5000.0 :   True
    oil3 :  -Inf : x[oil3,gas1] + x[oil3,gas2] + x[oil3,gas3] : 5000.0 :   True


$$
\sum_{i=1}^{3} \sum_{j=1}^{3} x_{ij} \leq 14000 
$$

In [13]:
m1.c3 = pyo.Constraint(expr = sum(m1.x[i,j] for i in m1.oil for j in m1.gas)<=14000)
m1.c3.pprint()

c3 : Size=1, Index=None, Active=True
    Key  : Lower : Body                                                                                                                                 : Upper   : Active
    None :  -Inf : x[oil1,gas1] + x[oil1,gas2] + x[oil1,gas3] + x[oil2,gas1] + x[oil2,gas2] + x[oil2,gas3] + x[oil3,gas1] + x[oil3,gas2] + x[oil3,gas3] : 14000.0 :   True


$$
\sum_{i=1}^{3} op_{i} x_{ij} \geq \ on_{j} \sum_{i=1}^{3} x_{ij} \quad \forall j
$$

In [14]:
m1.oktan_portion = pyo.Param(m1.oil , initialize ={'oil1':12,'oil2':6,'oil3':8})
m1.oktan_need = pyo.Param(m1.gas ,initialize ={'gas1':10,'gas2':8,'gas3':6} )

def oktan_constraint(model, j):
    return ( sum(model.oktan_portion[i] * model.x[i,j] for i in model.oil) >=
            model.oktan_need[j] * sum(model.x[i,j] for i in model.oil))
m1.c4 = pyo.Constraint(m1.gas , rule= oktan_constraint)
m1.c4.pprint()

c4 : Size=3, Index=gas, Active=True
    Key  : Lower : Body                                                                                                  : Upper : Active
    gas1 :  -Inf : 10*(x[oil1,gas1] + x[oil2,gas1] + x[oil3,gas1]) - (12*x[oil1,gas1] + 6*x[oil2,gas1] + 8*x[oil3,gas1]) :   0.0 :   True
    gas2 :  -Inf :  8*(x[oil1,gas2] + x[oil2,gas2] + x[oil3,gas2]) - (12*x[oil1,gas2] + 6*x[oil2,gas2] + 8*x[oil3,gas2]) :   0.0 :   True
    gas3 :  -Inf :  6*(x[oil1,gas3] + x[oil2,gas3] + x[oil3,gas3]) - (12*x[oil1,gas3] + 6*x[oil2,gas3] + 8*x[oil3,gas3]) :   0.0 :   True


$$
\sum_{i=1}^{3} sp_{i} x_{ij} \leq \ sn_{j} \sum_{i=1}^{3} x_{ij} \quad \forall j
$$

In [15]:
m1.solfor_portion = pyo.Param(m1.oil ,initialize ={'oil1':0.005,'oil2':0.02,'oil3':0.03})
m1.solfor_need    = pyo.Param(m1.gas ,initialize ={'gas1':0.01,'gas2':0.02,'gas3':0.01} )

def solfor_constraint(model, j):
    return ( sum(model.solfor_portion[i] * model.x[i,j] for i in model.oil) <=
            model.solfor_need[j] * sum(model.x[i,j] for i in model.oil))
m1.c5 = pyo.Constraint(m1.gas , rule= solfor_constraint)
m1.c5.pprint()

c5 : Size=3, Index=gas, Active=True
    Key  : Lower : Body                                                                                                           : Upper : Active
    gas1 :  -Inf : 0.005*x[oil1,gas1] + 0.02*x[oil2,gas1] + 0.03*x[oil3,gas1] - 0.01*(x[oil1,gas1] + x[oil2,gas1] + x[oil3,gas1]) :   0.0 :   True
    gas2 :  -Inf : 0.005*x[oil1,gas2] + 0.02*x[oil2,gas2] + 0.03*x[oil3,gas2] - 0.02*(x[oil1,gas2] + x[oil2,gas2] + x[oil3,gas2]) :   0.0 :   True
    gas3 :  -Inf : 0.005*x[oil1,gas3] + 0.02*x[oil2,gas3] + 0.03*x[oil3,gas3] - 0.01*(x[oil1,gas3] + x[oil2,gas3] + x[oil3,gas3]) :   0.0 :   True


In [18]:
# ! pip install highspy
solver_name = 'cplex'
solver = pyo.SolverFactory(solver_name)
result = solver.solve(m1)

In [19]:
print(result.solver.Termination_condition)

optimal


In [20]:
print(f'the optimal objective function = {pyo.value(m1.obj)}')

the optimal objective function = 126000.0


In [21]:
for i , j  in itertools.product(m1.oil,m1.gas):
    print(f'Value of x{i,j} = {np.round(pyo.value(m1.x[i,j]))}')

Value of x('oil1', 'gas1') = 2400.0
Value of x('oil1', 'gas2') = 800.0
Value of x('oil1', 'gas3') = 800.0
Value of x('oil2', 'gas1') = 0.0
Value of x('oil2', 'gas2') = 0.0
Value of x('oil2', 'gas3') = 0.0
Value of x('oil3', 'gas1') = 600.0
Value of x('oil3', 'gas2') = 1200.0
Value of x('oil3', 'gas3') = 200.0


In [22]:
for j in m1.gas:
    val = np.round(sum(pyo.value(m1.x[i,j]) for i in m1.oil))
    print(f'Value of {j} = {val}')
    print('-'*30)


Value of gas1 = 3000.0
------------------------------
Value of gas2 = 2000.0
------------------------------
Value of gas3 = 1000.0
------------------------------
